In [1]:
from lida import Manager
from lida.utils import read_dataframe

In [2]:
read_dataframe("test data sales-1.csv")

/Users/sawradip/miniconda3/envs/lida-env/lib/python3.11/site-packages/lida/utils.py:58: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  'csv': lambda: pd.read_csv(file_location, encoding=encoding),


,DepotName,Period,InvoiceDate,BusinessName,ProductCode,ProductName,BrandName,CustomerCode,CustomerName,DistrictName,...,Level2Name,Level3Name,Region,Base,BroadCategory,Latitude,Longitude,BusiSumGroupName,SubGroupName,L2Base
37081,Jashore,202301,1/11/2023,CC & PH,CCCM,Camamix 750 WP 100gm,Camamix 750 WP,NA0FIF,M/S Molla Traders,Narail,...,Vacant Jashore_AR,MD. MAMUNUR RASHID,Jashore,Narail,Retail,23.172534,89.512672,Camamix 750 WP,Fungicide,Jashore
16654,Dinajpur,202401,1/9/2024,CC & PH,AGGH,Fighter 2.5 Ec 500 Ml,FIGHTER,SA0B2X,RAJU TRADERS,Panchagarh,...,Abdur Rouf Prodhan,Md. Abdul Baki,Dinajpur,Debiganj,Retail,26.341100,88.554161,Fighter 2.5 EC,Liquid Insecticide,Thakurgaon
148227,Naogaon,202303,3/16/2023,CC & PH,AGFP,Care 50 Sp 100 Gm,CARE,AUA0L3A,Special 8 Cash Product (NAOGAON SADAR),NaN,...,Vacant Naogaon_AR,Md. Siddiqur Rahman,Naogaon,NAOGAON SADAR,Retail,NaN,NaN,Care 50 SP,Powder Insecticide,Naogaon
48591,Sreemongal,202402,2/3/2024,CC & PH,CC17,Supermix 18 WP 100gm,Supermix,PA0DO3,M/S Tarek Traders,Habiganj,...,Md. Ahid Uddin Bhuiyan,Md. Ahid Uddin Bhuiyan,Chattogram,Habiganj,Retail,24.374945,91.415530,Supermix 18 WP,Herbicide,Chattogram
81167,Barishal,202308,8/20/2023,CC & PH,AGWL,AC Mix 55 EC 50ml,AC Mix 55 EC,LA00T4,M/S MASTER TRADERS,Bhola,...,Md. Touhidul Isalm,Md. Al Amin,Barishal,Lalmohan,Retail,22.685923,90.648179,ACmix 55 EC,Liquid Insecticide,Barishal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25951,Barishal,202302,2/18/2023,CC & PH,AGOF,Weednil 5 EC 100ml,Weednil 5 EC,LA0H32,M/s Priyalal Store,Madaripur,...,Vacant Barishal_AR,Md. Al Amin,Barishal,Shibchar,Retail,23.164102,90.189680,Weednil 5 EC,Herbicide,Kushtia
178617,Chattogram,202312,12/29/2023,CC & PH,CC68,Paraxone 20 SL,PARAXONE,AA03PC,BISMILLAH AGRO SEED (TA03PC),Coxs Bazar,...,Vacant Chattogram_AR,Md. Mehediul Alam,Chattogram,Coxsbazar,Retal,21.450974,91.967883,Paraxone 20 SL,Herbicide,Chattogram
41475,Rajshahi,202304,4/12/2023,CC & PH,AGWL,AC Mix 55 EC 50ml,AC Mix 55 EC,IA0FZ6,M/S KHAN TRADERS,Pabna,...,Md. Rezaul Karim,Md. Halimuzzaman,Rajshahi,Pabna,Retal,23.998524,89.233645,ACmix 55 EC,Liquid Insecticide,Pabna
88527,Chattogram,202306,6/8/2023,CC & PH,CCCF,Pyrazin 70 WDG,Pyrazin 70 WDG,AA0JC3,M/S. HAZI LAL MIA & SONS,Chittagong,...,Vacant Chattogram_AR,Md. Mehediul Alam,Chattogram,Raozan,Retal,22.335109,91.834073,Pyrazin 70 WDG,Powder Insecticide,Chattogram


# Utils

### Summary

> <b>Topic1</b>: text <br>
> <b>Topic1</b>: text <br>
> <b>Topic1</b>: text <br>

In [56]:
MAX_COLUMNS = 100
MAX_GOALS = 10
MAX_EXPLANATIONS = 10
MAX_VISUALIZATIONS=1
SUMMARY_LLM_MODEL = "gpt-3.5-turbo-16k" #"gpt-4o"
TASK_LLM_MODEL = "gpt-4o"

NON_MINIMAL_MODE = True

import os
import gradio as gr
from lida import Manager
from lida.utils import read_dataframe
from lida.datamodel import TextGenerationConfig, Goal
from llmx import TextGenerator
from dotenv import load_dotenv
from PIL import Image
from io import BytesIO
import base64
load_dotenv(".env")

lida_manager = Manager()

def display_dataframe(data_filepath, row_count = 5):
    df = read_dataframe(data_filepath)
    # print(df)
    df_viewer = gr.DataFrame(data_filepath, row_count = row_count, visible=True)
    selected_col_list = gr.CheckboxGroup(
        df.columns.tolist(),
        value=df.columns.tolist(),
        visible=True,
        label="Select Columns",
        info="The ones you want to analyze.")
    return df_viewer, selected_col_list

def modify_dataframe(df_viewer, col_list):
    df_viewer = df_viewer[col_list]
    return df_viewer


def get_summary(data_filepath, textgen_config=None):
    global lida_manager
    data_summary = lida_manager.summarize(
        data_filepath,
        # text_gen: TextGenerator, 
        file_name="", 
        n_samples = 3,
        textgen_config=textgen_config if textgen_config is not None else TextGenerationConfig(model=SUMMARY_LLM_MODEL),
        summary_method = "llm",
        )
    
    return data_summary


def display_summary(data_filepath, ):
    global lida_manager

    data_summary = get_summary(data_filepath, textgen_config=None)
    
    # print(data_summary)
    name = data_summary["name"]
    filename =  data_summary["file_name"]
    data_description =   data_summary["dataset_description"]
    fields = data_summary["fields"]

    data_info_str = f"## {name}\n"
    data_info_str += f"### File: {filename}\n"
    data_info_str += f"### {data_description}\n"

    data_info = gr.Markdown(data_info_str)

    # column_descriptors = []
    summary_full_str = ""
    
    for field in fields:
        field_name = field['column']
        field_properties = field['properties']
        field_dtype = field_properties.pop("dtype")
        field_description = field_properties.pop("description")

        field_header = f'<br><h3> {field_name} ({field_dtype}) - {field_description}</h3>'

        field_details = []
        for detail_name, detail_value in field_properties.items():
            field_details.append(f"> {detail_name}: {', '.join(map(str, detail_value)) if isinstance(detail_value, list) else str(detail_value)}")

        field_details_str = "<br>".join(field_details)

        summary_full_str += field_header
        summary_full_str += field_details_str

    
    summary_descriptor = gr.Markdown(summary_full_str, visible=True)

    #     # print(field_details_str)
    #     with gr.Accordion(, open=False, visible=True) as column_descriptor:
    #         column_details = gr.Markdown(field_details_str)       
             
    #     column_descriptors.append(column_descriptor)
    #     column_descriptors.append(column_details)

    # for i in range(MAX_COLUMNS - len(fields)):
    #     with gr.Accordion("", open=False, visible=False) as column_descriptor:
    #         column_details = gr.Markdown("")

    #         column_descriptors.append(column_descriptor)
    #     column_descriptors.append(column_details)

    return [data_info, summary_descriptor]

def base64_to_image(base64_string):
    # Decode the base64 string
    byte_data = base64.b64decode(base64_string)
    
    # Use BytesIO to convert the byte data to image
    return Image.open(BytesIO(byte_data))

In [57]:
data_filepath = "test data sales-1.csv"

display_dataframe(data_filepath, row_count = 5)

/Users/sawradip/miniconda3/envs/lida-env/lib/python3.11/site-packages/lida/utils.py:58: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  'csv': lambda: pd.read_csv(file_location, encoding=encoding),
/Users/sawradip/miniconda3/envs/lida-env/lib/python3.11/site-packages/gradio/components/dataframe.py:257: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  value = pd.read_csv(value)  # type: ignore


(<gradio.components.dataframe.Dataframe at 0x30f051b90>,
 <gradio.components.checkboxgroup.CheckboxGroup at 0x16b5dae90>)

In [58]:
def display_goals(data_filepath, n_goals, insight_persona):
    global lida_manager

    data_summary = get_summary(data_filepath, textgen_config=None)

    goals = lida_manager.goals(data_summary, n=n_goals, persona=insight_persona)

    goal_cards = []
    for goal in goals:
        with gr.Accordion(f"{goal.question}", open=False, visible=True) as goal_card:
            question_holder = gr.Text(goal.question)
            visualization_holder = gr.Text(goal.visualization)
            rationale_holder = gr.Text(goal.rationale)
            visualization_text = gr.Markdown(f"""*<span style="color: grey;"><b>Visualization:<b>{goal.visualization}</span>*""")
            with gr.Row(equal_height=True):   
                with gr.Column(scale=3):
                    rationale_text = gr.Markdown(f"**Rationale:** {goal.rationale}")
                generate_button = gr.Button("Visualize", scale=1, visible=True)

        goal_card_components = [goal_card, 
                                question_holder, 
                                visualization_holder, 
                                rationale_holder, 
                                visualization_text, 
                                rationale_text, 
                                generate_button, 
                                # *visualization_blocks
                                ]
        goal_cards.extend(goal_card_components)


    for i in range(MAX_GOALS - len(goals)):
        with gr.Accordion("", open=False, visible=False) as goal_card:
            question_holder = gr.Textbox("", visible=False)
            visualization_holder = gr.Textbox("", visible=False)
            rationale_holder = gr.Textbox("", visible=False)
            visualization_text = gr.Markdown("",)
            with gr.Row(equal_height=True):   
                with gr.Column(scale=3):
                    rationale_text = gr.Markdown("")
                generate_button = gr.Button(scale=1, visible=False)

        goal_card_components = [goal_card, 
                                question_holder, 
                                visualization_holder, 
                                rationale_holder, 
                                visualization_text, 
                                rationale_text, 
                                generate_button, 
                                # *visualization_blocks
                                ]
        goal_cards.extend(goal_card_components)
    return goal_cards

def display_visualization(data_filepath, goal_question, goal_visualization, goal_rationale):
    print("display_visualization clicked")
    global lida_manager

    data_summary = get_summary(data_filepath, textgen_config=None)
    
    goal = Goal(question = goal_question,
                visualization = goal_visualization,
                rationale = goal_rationale)

    charts = lida_manager.visualize(summary=data_summary, goal=goal, library="matplotlib")

    visualization_pieces = []
    for chart in charts:
        # if chart.status and not chart.error:
        # with gr.Row(equal_height=True): 
        #     chart_img = gr.Image(base64_to_image(chart.raster), visible=True) 
        #     code_block = gr.Code(chart.code, language="python", visible=True)

        # visualization_pieces.append(chart_img)
        # visualization_pieces.append(code_block)

        # with gr.Row(equal_height=True): 
        chart_img = gr.Image(base64_to_image(chart.raster), visible=True)
        with gr.Accordion("Code Here", visible=NON_MINIMAL_MODE) as code_holder:
            code_block = gr.Code(chart.code, language="python", visible=True)
        visualization_pieces.append(chart_img)
        visualization_pieces.append(code_holder)
        visualization_pieces.append(code_block)


    for i in range(MAX_VISUALIZATIONS - len(charts)):
        # with gr.Row(equal_height=True): 
        #     chart_img = gr.Image(visible=False) 
        #     code_block = gr.Code(visible=False)
            
        # visualization_pieces.append(chart_img)
        # visualization_pieces.append(code_block)
    
        chart_img = gr.Image(visible=False)
        with gr.Accordion("Code Here", visible=False) as code_holder:
            code_block = gr.Code(language="python", visible=False)
        visualization_pieces.append(chart_img)
        visualization_pieces.append(code_holder)
        visualization_pieces.append(code_block)

    with gr.Row():
        modification_text = gr.Textbox("", label="Write expected modifications(seperated by commma)", interactive=True, visible=True, scale=3)
        modify_button = gr.Button("Modify Visualizations", scale=1, visible=True)
    recommendation_button = gr.Button("Show Similar Visualizations", scale=0.5, visible=True)

    return visualization_pieces + [modification_text, modify_button, recommendation_button]
            

# instructions = ["convert this to a bar chart", "change the theme to red", "translate the title to Bangla"]
# edited_charts = lida.edit(code=charts[0].code,  summary=summary, instructions=instructions, library='seaborn', 
#                         # textgen_config=textgen_config
#                         )

def display_modified_visualization(data_filepath, existing_code, modification_instructions):
    print("display_modified_visualization clicked")
    global lida_manager

    data_summary = get_summary(data_filepath, textgen_config=None)

    modification_instruction_list = modification_instructions.split(",")

    edited_charts = lida_manager.edit(code=existing_code,  
                                summary=data_summary, 
                                instructions=modification_instruction_list, 
                                library='seaborn', 
                                textgen_config=TextGenerationConfig(model=TASK_LLM_MODEL),
                                )
    print(edited_charts)
    visualization_pieces = []
    for chart in edited_charts:
        # if chart.status and not chart.error:
        # with gr.Row(equal_height=True): 
            # chart_img = gr.Image(base64_to_image(chart.raster), visible=True) 
            # code_block = gr.Code(chart.code, language="python", visible=True)
        # visualization_pieces.append(chart_img)
        # visualization_pieces.append(code_block)

        chart_img = gr.Image(base64_to_image(chart.raster), visible=True)
        with gr.Accordion("Code Here", visible=NON_MINIMAL_MODE) as code_holder:
            code_block = gr.Code(chart.code, language="python", visible=True)
        visualization_pieces.append(chart_img)
        visualization_pieces.append(code_holder)
        visualization_pieces.append(code_block)



    for i in range(MAX_VISUALIZATIONS - len(edited_charts)):
        # with gr.Row(equal_height=True): 
        #     chart_img = gr.Image(visible=False) 
        #     code_block = gr.Code(visible=False)
            
        # visualization_pieces.append(chart_img)
        # visualization_pieces.append(code_block)
        chart_img = gr.Image(visible=False)
        with gr.Accordion("Code Here", visible=False) as code_holder:
            code_block = gr.Code(language="python", visible=False)
        visualization_pieces.append(chart_img)
        visualization_pieces.append(code_holder)
        visualization_pieces.append(code_block)
        
    print("display_modified_visualization exited")
    return visualization_pieces



def display_recommendation_visualization(data_filepath, existing_code, n = 5):
    print("display_recommendation_visualization clicked")

    global lida_manager

    data_summary = get_summary(data_filepath, textgen_config=None)

    recommendations = lida_manager.recommend(existing_code, 
                                    summary=data_summary, 
                                    n=n,  
                                    textgen_config=TextGenerationConfig(model=TASK_LLM_MODEL),
                                    )

    recommendation_gallery = gr.Gallery(
                                    [base64_to_image(recommendation.raster) for recommendation in recommendations],
                                    visible=True, 
                                    preview=True,
                                    # allow_preview = False            
                                        )

    return recommendation_gallery

In [59]:

def display_visualization_only_question(data_filepath, goal_question):
    print("visualization_only_question clicked")
    global lida_manager

    data_summary = get_summary(data_filepath, textgen_config=None)

    # goal = Goal(question = goal_question,
    #         visualization = "",
    #         rationale = "")
    charts = lida_manager.visualize(
        summary=data_summary, 
        goal=goal_question, 
        textgen_config=TextGenerationConfig(model=TASK_LLM_MODEL),
        library="seaborn")

    visualization_pieces = []
    for chart in charts:
        # # if chart.status and not chart.error:
        # with gr.Row(equal_height=True): 
        #     chart_img = gr.Image(base64_to_image(chart.raster), visible=True, scale=2) 
        #     code_block = gr.Code(chart.code, language="python", visible=True, scale=1) 

        # visualization_pieces.append(chart_img)
        # visualization_pieces.append(code_block)
        chart_img = gr.Image(base64_to_image(chart.raster), visible=True)
        with gr.Accordion("Code Here", visible=NON_MINIMAL_MODE) as code_holder:
            code_block = gr.Code(chart.code, language="python", visible=True)
        visualization_pieces.append(chart_img)
        visualization_pieces.append(code_holder)
        visualization_pieces.append(code_block)

    for i in range(MAX_VISUALIZATIONS - len(charts)):
        # with gr.Row(equal_height=True): 
        #     chart_img = gr.Image(visible=False, scale=2) 
        #     code_block = gr.Code(visible=False, scale=1) 
            
        # visualization_pieces.append(chart_img)
        # visualization_pieces.append(code_block)
        chart_img = gr.Image(visible=False)
        with gr.Accordion("Code Here", visible=False) as code_holder:
            code_block = gr.Code(language="python", visible=False)
        visualization_pieces.append(chart_img)
        visualization_pieces.append(code_holder)
        visualization_pieces.append(code_block)

    with gr.Row():
        modification_text = gr.Textbox("", label="Write expected modifications(seperated by commma)", interactive=True, visible=True, scale=3)
        modify_button = gr.Button("Modify Visualizations", scale=1, visible=True)
    
    with gr.Row():
            explanation_button = gr.Button(visible=True)
            evaluation_button = gr.Button(visible=True)
            recommendation_button = gr.Button(visible=True)


    print("visualization_only_question exiting")
    return visualization_pieces + [ modification_text, 
                                    modify_button, 
                                    explanation_button, 
                                    evaluation_button,
                                    recommendation_button,
                                    ]

def display_explanation(data_filepath, existing_code):
    print("visualization explanation clicked")
    global lida_manager

    print(existing_code)
    data_summary = get_summary(data_filepath, textgen_config=None)
    
    print("### Explanation Code Sending:", existing_code)
    explanation_items = lida_manager.explain(code=existing_code)[0]
    print("### Explanation Recieved:", explanation_items)

    explanation_evaluation_blocks = []
    for i, explanation_item in enumerate(explanation_items):
        section = explanation_item["section"]
        code = explanation_item["code"]
        explanation = explanation_item["explanation"]
        print("#####", i, code)
        with gr.Accordion(f"Code Block {i+1}", visible=True) as explanator_evaluator:
            code_block = gr.Code(code, visible=True)
            small_text1 = gr.Markdown(f"<b>Section</b>: {section}")
            small_text2 = gr.Markdown(f"<b>Explanation</b>: {explanation}")

        explanation_evaluation_blocks.append(explanator_evaluator)
        explanation_evaluation_blocks.append(code_block)
        explanation_evaluation_blocks.append(small_text1)
        explanation_evaluation_blocks.append(small_text2)
        
    for _ in range(MAX_EXPLANATIONS - len(explanation_items)):
        with gr.Accordion("", visible=False) as explanator_evaluator:
            code_block = gr.Code(visible=False)
            small_text1 = gr.Markdown("")
            small_text2 = gr.Markdown("")

        explanation_evaluation_blocks.append(explanator_evaluator)
        explanation_evaluation_blocks.append(code_block)
        explanation_evaluation_blocks.append(small_text1)
        explanation_evaluation_blocks.append(small_text2)

    return explanation_evaluation_blocks


def display_evaluation(data_filepath, existing_code,goal_question):
    print("visualization explanation clicked")
    global lida_manager

    data_summary = get_summary(data_filepath, textgen_config=None)

    goal = Goal(question = goal_question,
                visualization = goal_question,
                rationale = "")
    explanation_items = lida_manager.evaluate(code=existing_code, goal = goal)[0]

    explanation_evaluation_blocks = []
    for i, explanation_item in enumerate(explanation_items):
        rationale = explanation_item["rationale"]
        dimension = explanation_item["dimension"]
        score = explanation_item["score"]
        with gr.Accordion(rationale, visible=True) as explanator_evaluator:
            code_block = gr.Code(visible=False)
            small_text1 = gr.Markdown(f"<b>Score</b>: {score}")
            small_text2 = gr.Markdown(f"<b>Dimension</b>: {dimension}")

        explanation_evaluation_blocks.append(explanator_evaluator)
        explanation_evaluation_blocks.append(code_block)
        explanation_evaluation_blocks.append(small_text1)
        explanation_evaluation_blocks.append(small_text2)
        
    for _ in range(MAX_EXPLANATIONS - len(explanation_items)):
        with gr.Accordion("", visible=False) as explanator_evaluator:
            code_block = gr.Code(visible=False)
            small_text1 = gr.Markdown("")
            small_text2 = gr.Markdown("")

        explanation_evaluation_blocks.append(explanator_evaluator)
        explanation_evaluation_blocks.append(code_block)
        explanation_evaluation_blocks.append(small_text1)
        explanation_evaluation_blocks.append(small_text2)

    return explanation_evaluation_blocks

# UI

In [60]:
import gradio as gr

title_html = (
    """<br><h1 style="text-align:center; font-size: 50px; """
    """text-shadow: -1px 0 white, 0 1px white, 1px 0 white, 0 -1px white;"> """
    """<span style="color:#000000;">Tabular</span>"""
    """<span style="color:#6bc7f0;">Explorer</span> </h1>"""
)
subtitle_html = (
    """<h2 style="text-align:center; """
    """text-shadow: -1px 0 white, 0 1px white, 1px 0 white, 0 -1px white;"> """
    """<span style="color:#000000;">Talk to your data files — """
    """Get Insights</span></h2>"""
)

with gr.Blocks(
    css="footer{display:none !important}"
) as demo:
    gr.HTML(title_html)
    gr.HTML(subtitle_html)
    # name = gr.Textbox(label="OpenAI API Key", type= "password")
    data_file = gr.File(height="0.5in")
    selected_col_list = gr.CheckboxGroup(visible=False, label="Select Columns", info="The ones you want to analyze.")
    data_viewer = gr.DataFrame(
        visible=False,
        label = "Loaded Data",
        show_label=True)
    
    data_file.upload(
        display_dataframe,
        inputs=[data_file],
        outputs=[data_viewer, selected_col_list]
        )
    selected_col_list.change(
        modify_dataframe,
        inputs=[data_viewer, selected_col_list],
        outputs=[data_viewer]

    )
    with gr.Tab("Summary"):
        show_summary_btn = gr.Button("Show Summary", interactive=True)
        data_info = gr.Markdown()
        summary_descriptor = gr.Markdown()

        show_summary_btn.click( display_summary,
                            inputs=[data_viewer],
                            outputs=[
                                data_info,
                                summary_descriptor
                                ]
                            )
    with gr.Tab("Insights"):
        with gr.Row():
            persona_txt = gr.Text("Business Analyst at at large Corporate Organization.", interactive=True, scale=3)
            goal_counter = gr.Slider(minimum=2, maximum=10, step=1, interactive=True, scale=1)
        
        propose_insights_btn = gr.Button("Propose Insights")


        goal_cards = []
        for i in range(MAX_GOALS):
            with gr.Accordion("", visible=NON_MINIMAL_MODE) as goal_card:
                question_holder = gr.Textbox("", visible=False)
                visualization_holder = gr.Textbox("", visible=False)
                rationale_holder = gr.Textbox("", visible=False)
                visualization_text = gr.Markdown("",)
                with gr.Row(equal_height=True):   
                    with gr.Column(scale=3):
                        rationale_text = gr.Markdown("")
                    visualize_button = gr.Button(scale=1, visible=False)

                visualization_blocks = []
                for n_row in range(MAX_VISUALIZATIONS):
                    # with gr.Row(equal_height=True): 
                    chart_img = gr.Image(visible=False, scale=2)
                    with gr.Accordion("", visible=NON_MINIMAL_MODE) as code_holder:
                        code_block = gr.Code(visible=False, scale=1) 
                    visualization_blocks.append(chart_img)
                    visualization_blocks.append(code_holder)
                    visualization_blocks.append(code_block)
                with gr.Row():
                    modification_text = gr.Textbox("", visible=False, scale=3)
                    modify_button = gr.Button(scale=1, visible=False)
                recommendation_button = gr.Button(scale=1, visible=False)

                recommendation_gallery = gr.Gallery(visible=False)

            visualize_button.click(
                        display_visualization,
                        inputs = [data_file, question_holder, visualization_holder, rationale_holder],
                        outputs = visualization_blocks + [modification_text, modify_button, recommendation_button]
            )
            modify_button.click(
                        display_modified_visualization,
                        inputs = [data_file, code_block, modification_text],
                        outputs = visualization_blocks
            )

            recommendation_button.click(
                display_recommendation_visualization,
                inputs = [data_file, code_block],
                outputs = [recommendation_gallery]
            )

            goal_card_components = [goal_card, 
                                    question_holder, 
                                    visualization_holder, 
                                    rationale_holder, 
                                    visualization_text, 
                                    rationale_text, 
                                    visualize_button, 
                                    # *visualization_blocks,
                                    ]
            goal_cards.extend(goal_card_components)

        
    with gr.Tab("Queries"):
        with gr.Row():
            query_txt = gr.Text(interactive=True, scale=3)
            query_btn = gr.Button("Ask or Query")

        visualization_blocks = []
        for n_row in range(MAX_VISUALIZATIONS):
            chart_img = gr.Image(visible=False, scale=2)
            with gr.Accordion("Code Here", visible=NON_MINIMAL_MODE) as code_holder:
                code_block = gr.Code(visible=False, scale=1) 
            visualization_blocks.append(chart_img)
            visualization_blocks.append(code_holder)
            visualization_blocks.append(code_block)

        with gr.Row():
            modification_text = gr.Textbox("", visible=False, scale=3)
            modify_button = gr.Button(scale=1, visible=False)

        with gr.Row():
            recommendation_button = gr.Button("Similar Plots", scale=3, visible=False)
            evaluation_button = gr.Button("Analyse Answer", scale=1, visible=False)
            explanation_button = gr.Button("Explain Code", scale=1, visible=False)

        recommendation_gallery = gr.Gallery(visible=False)

        explanation_evaluation_blocks = []
        for _ in range(MAX_EXPLANATIONS):
            with gr.Accordion("", visible=False) as explanator_evaluator:
                code_subblock = gr.Code(visible=False)
                small_text1 = gr.Markdown("")
                small_text2 = gr.Markdown("")

            explanation_evaluation_blocks.append(explanator_evaluator)
            explanation_evaluation_blocks.append(code_subblock)
            explanation_evaluation_blocks.append(small_text1)
            explanation_evaluation_blocks.append(small_text2)

        query_btn.click(
            display_visualization_only_question,
            inputs = [data_file, query_txt],
            outputs = visualization_blocks + [ modification_text, modify_button, explanation_button, 
                                                evaluation_button, recommendation_button]
        )

        modify_button.click(
            display_modified_visualization,
            inputs = [data_file, code_block, modification_text],
            outputs = visualization_blocks
        )

        recommendation_button.click(
            display_recommendation_visualization,
            inputs = [data_file, code_block],
            outputs = [recommendation_gallery]
        )

        explanation_button.click(
            display_explanation,
            inputs = [data_file, code_block],
            outputs = explanation_evaluation_blocks
        )

        evaluation_button.click(
            display_evaluation,
            inputs = [data_file, code_block, query_txt],
            outputs = explanation_evaluation_blocks
        )
                            
    propose_insights_btn.click(display_goals,
                            inputs = [
                                data_viewer,
                                goal_counter,
                                persona_txt],
                            outputs = goal_cards
                            )


    with gr.Tab("Index"):
        with gr.Row():
            dropdown1 = gr.Dropdown(choices=["Option 1", "Option 2", "Option 3"], label="Dropdown 1", interactive=True)
            dropdown2 = gr.Dropdown(choices=["Option A", "Option B", "Option C"], label="Dropdown 2", interactive=True)
            dropdown3 = gr.Dropdown(choices=["Choice X", "Choice Y", "Choice Z"], label="Dropdown 3", interactive=True)

            query_txt = gr.Text(interactive=True, scale=3)
            query_btn = gr.Button("Ask or Query")

demo.launch( share=False, 
            debug=True,
            # server_port = 7860
            )

# Distribution of arrived people. 

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


display_visualization clicked


/Users/sawradip/miniconda3/envs/lida-env/lib/python3.11/site-packages/gradio/components/base.py:190: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(


display_modified_visualization clicked
[ChartExecutorResponse(spec=None, status=True, raster='...', code="import seaborn as sns\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# solution plan\n# i. Group data by 'Business' and sum 'TP'\n# ii. Plot a bar chart of total sales by business\n# iii. Add a vertical line for the maximum sales value with a legend\ndef plot(data: pd.DataFrame):\n    business_sales = data.groupby('Business')['TP'].sum().reset_index()\n    max_sales = business_sales['TP'].max()\n    \n    plt.figure(figsize=(10, 6))\n    sns.barplot(x='Business', y='TP', data=business_sales)\n    plt.axhline(max_sales, color='r', linestyle='--', label=f'Max Sales: {max_sales:.2f}')\n    plt.xlabel('Business')\n    plt.ylabel('Total Sales')\n    plt.title('Distribution of Sales by Business')\n    plt.legend()\n    \n    return plt\n\nchart = plot(data)", library='seaborn', error=None)]
display_modified_visualization exited
display_visualization clicked


/Users/sawradip/miniconda3/envs/lida-env/lib/python3.11/site-packages/gradio/components/base.py:190: UserWarning: 'scale' value should be an integer. Using 0.5 will cause issues.
  warnings.warn(
